In [49]:
import pyspark
from pyspark.sql import SparkSession, Row, SQLContext
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from datetime import datetime, timedelta
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, sum
from datetime import datetime, timedelta
import os

In [50]:


spark = SparkSession\
    .builder\
    .master("local[4]")\
    .appName("sparkJob2")\
    .config("spark.eventLog.logBlockUpdates.enabled", True)\
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

In [46]:
#add the path for the hive table on hdfs 


from pyspark.sql.functions import col, current_date, date_sub, to_date
tranformed_data = spark.read.orc("/user/hive/warehouse/transactionsdb.db/transactions_table/*")
tranformed_data.count()




557500

In [47]:

# Convert the 'transaction_date' to date type (assuming it's in 'yyyy-MM-dd' format, adjust format as needed)
transformed_data = tranformed_data.withColumn("transaction_date", to_date(col("transaction_date"), "yyyy-MM-dd"))

# Define the date 1.5 years ago from today
one_and_half_years_ago = date_sub(current_date(), int(1.5 * 365)+1)  # 1.5 years in days

# Filter the data where transaction_date is within the last 1.5 years
filtered_data = tranformed_data.filter(col("transaction_date") == one_and_half_years_ago)

filtered_data.count()



824

In [51]:
# Show the filtered DataFrame
filtered_data.show(truncate=False)

+----------------+----------------+-----------+--------------+--------------+---------------------------+--------------+---------+----------+-----------------+----------------+-------+-------+-------+-------+-------+-----+----------+---------+--------------+----------------+----+------+---------+------------------+---------+----+-----+
|transaction_date|transaction_id  |customer_id|customer_fname|cusomter_lname|cusomter_email             |sales_agent_id|branch_id|product_id|product_name     |product_category|offer_1|offer_2|offer_3|offer_4|offer_5|units|unit_price|is_online|payment_method|shipping_address|logs|source|discounts|total_paid        |city     |Year|Month|
+----------------+----------------+-----------+--------------+--------------+---------------------------+--------------+---------+----------+-----------------+----------------+-------+-------+-------+-------+-------+-----+----------+---------+--------------+----------------+----+------+---------+------------------+--------

In [64]:
from datetime import datetime, timedelta

# Function to get the previous day's date and the hour from 23 hours ago
def get_last_hour():
    now = datetime.now()
    last_day = now - timedelta(days=1)  # Subtract one day
    return last_day.strftime("%Y-%m-%d"), "23"  # Return previous day and hour 23

# Get the date and hour for the last hour
target_date, target_hour = get_last_hour()

# Base directory where your data is stored
base_dir = "/finalData/spark_project/sales_agents2"
dynamic_dir = f"date({target_date})"

# Full path to the directory
directory_path = os.path.join(base_dir, dynamic_dir)

# Print the directory path for debugging
print("Directory Path:", directory_path)

# Use glob to list all files in the directory
import glob

file_pattern = os.path.join(directory_path, '*')
all_files = glob.glob(file_pattern)




Directory Path: /finalData/spark_project/sales_agents2/date(2024-07-04)


In [65]:
agents = spark.read.csv(directory_path, header=True)
agents.count()

63

In [66]:
agents = agents.dropDuplicates(["sales_person_id"])
agents.count()

11

In [67]:
# Perform join and aggregation
result_df = tranformed_data.join(agents, tranformed_data.sales_agent_id == agents.sales_person_id, "inner") \
                          .groupBy("name", "product_name") \
                          .agg(sum("units").alias("total_sold_units")) \
                          .select("name", "product_name", "total_sold_units")


In [68]:
result_df.show()
result_df.count()

+------------------+-----------------+----------------+
|              name|     product_name|total_sold_units|
+------------------+-----------------+----------------+
|        Jane Smith|           Hoodie|          8664.0|
|          John Doe|             Iron|          8505.0|
|   Michael Johnson|               TV|          8310.0|
|          John Doe|          Sandals|         16600.0|
|      Sophia Moore|  Electric Kettle|          8338.0|
|      Olivia Davis|           Blouse|          8182.0|
|       Emily Brown|        Microwave|          8400.0|
|          John Doe|            Heels|          8364.0|
|Christopher Miller|            Heels|          8567.0|
|      David Wilson|  Washing Machine|          8389.0|
|Christopher Miller|          Monitor|          8239.0|
|       Emma Taylor|            Skirt|          8510.0|
|          John Doe|            Boots|         16591.0|
|Christopher Miller|Hair Straightener|          8374.0|
|   Michael Johnson|           Hoodie|          

308

In [69]:
df = result_df.coalesce(1)


In [70]:
# Write the DataFrame to a local CSV file with a header
df.write.option("header", "true").csv("file:///data/daily_dumb/daily_dumb.csv")
